<a href="https://colab.research.google.com/github/avikalsagar/Predicting_Annual_Variability_Renewable_Energy/blob/main/ERA5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#pip install earthengine-api

In [20]:
import ee
import pandas as pd

In [21]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=D_4eAdPMTo3uopUBESWQEzFmeoB6xExpH26uy6bA9lA&tc=9H-zDnV9HD9jtTVHZGiKbeM3uvF6sutjdLUXw6-imj4&cc=fQfwW2xR7kVuEVxI4RO2Esiduh55A7mwV5n_6eliZYM

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXn7SpPYekACtcRaRY5mSYMhNvWnQxJrzEiBWMcD1RDH40OPQsJCcEM

Successfully saved authorization token.


In [22]:
# Initialize Earth Engine
ee.Initialize()

In [53]:
# Define the region of interest for South Dakota
roi = ee.Geometry.Polygon([[[-104.5, 49],
                             [-97.20, 49],
                             [-100, 34.55],
                             [-94.44, 34.55]]])

In [54]:
# Load the ERA5-Land image collection
collection = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")

In [55]:
# Define the start and end dates
start_date = '2018-01-01'
end_date = '2018-01-02'

In [56]:
# Filter the collection based on location and time
filtered_collection = collection.filterBounds(roi).filterDate(start_date, end_date)

In [57]:
# Select the desired bands
band_names = [
    "temperature_2m",
    "surface_net_solar_radiation",
    "u_component_of_wind_10m",
    "v_component_of_wind_10m",
    "total_precipitation"
]

In [58]:
# Function to extract the desired bands from an image
def extract_bands(image):
    return image.select(band_names)

In [59]:
# Map over the image collection and apply the band extraction function
extracted_collection = filtered_collection.map(extract_bands)

In [60]:
# Get the image collection size
collection_size = extracted_collection.size().getInfo()

In [61]:
# Initialize an empty list to store the extracted band data
data = []

In [62]:
# Iterate over the image collection
for i in range(collection_size):
    image = ee.Image(extracted_collection.toList(collection_size).get(i))

    # Convert the image timestamp to milliseconds and then to a Python datetime object
    timestamp_ms = image.date().getInfo()["value"]
    date_time = pd.to_datetime(timestamp_ms, unit="ms")

    band_values = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=1000
    )

    # Create a new dictionary to store values, including the date
    data_point = {'date_time': date_time}
    data_point.update(band_values.getInfo())

    # Append the dictionary to the data list
    data.append(data_point)


In [63]:
# Convert the extracted band data to a pandas DataFrame
df = pd.DataFrame(data)
df

,date_time,surface_net_solar_radiation,temperature_2m,total_precipitation,u_component_of_wind_10m,v_component_of_wind_10m
0,2018-01-01 00:00:00,5.255214e+06,254.869501,2.311688e-04,1.093489,-2.243693
1,2018-01-01 01:00:00,0.000000e+00,254.192533,4.440347e-08,1.074556,-2.248866
2,2018-01-01 02:00:00,0.000000e+00,253.602073,4.542239e-08,1.023605,-2.209089
3,2018-01-01 03:00:00,0.000000e+00,253.110512,4.542239e-08,0.942507,-2.139801
4,2018-01-01 04:00:00,0.000000e+00,252.674437,4.536146e-08,0.921040,-2.013623
5,2018-01-01 05:00:00,0.000000e+00,252.256571,4.536503e-08,0.947823,-1.852341
6,2018-01-01 06:00:00,0.000000e+00,251.885023,4.551324e-08,0.992229,-1.709990
7,2018-01-01 07:00:00,0.000000e+00,251.097963,4.870371e-08,1.270438,-1.647403
8,2018-01-01 08:00:00,0.000000e+00,250.949146,5.207905e-08,1.154960,-1.661940
9,2018-01-01 09:00:00,0.000000e+00,250.923151,5.657386e-08,0.980637,-1.646764


In [ ]:
df.to_csv("2018 ERA5 data.csv")